In [1]:
import pandas as pd

import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)

uri = "https://gist.githubusercontent.com/guilhermesilveira/e99a526b2e7ccc6c3b70f53db43a87d2/raw/1605fc74aa778066bf2e6695e24d53cf65f2f447/machine-learning-carros-simulacao.csv"
dados = pd.read_csv(uri).drop(columns=["Unnamed: 0"], axis=1)
dados.head()

preco  vendido  idade_do_modelo   km_por_ano
0   30941.02        1               18  35085.22134
1   40557.96        1               20  12622.05362
2   89627.50        0               12  11440.79806
3   95276.14        0                3  43167.32682
4  117384.68        1                4  12770.11290

In [2]:
x = dados[["preco", "idade_do_modelo", "km_por_ano"]]
y = dados["vendido"]

In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.svm import LinearSVC
from sklearn.metrics import accuracy_score

x = dados[["preco", "idade_do_modelo","km_por_ano"]]
y = dados["vendido"]

SEED = 158020
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

Treinaremos com 7500 elementos e testaremos com 2500 elementos


In [4]:
from sklearn.dummy import DummyClassifier

dummy_stratified = DummyClassifier()
dummy_stratified.fit(treino_x, treino_y)
acuracia = dummy_stratified.score(teste_x, teste_y) * 100

print("A acurácia do dummy stratified foi de %.2f%%" % acuracia)

A acurácia do dummy stratified foi de 50.96%


In [5]:
from sklearn.tree import DecisionTreeClassifier


x = dados[["preco", "idade_do_modelo","km_por_ano"]]
y = dados["vendido"]

modelo = DecisionTreeClassifier(max_depth=2)

SEED = 158020
np.random.seed(SEED)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print ("A acurácia foi %.2f%%" % acuracia)

A acurácia foi 71.92%


In [6]:
x = dados[["preco", "idade_do_modelo","km_por_ano"]]
y = dados["vendido"]

SEED = 5
np.random.seed(SEED)
treino_x, teste_x, treino_y, teste_y = train_test_split(x, y, test_size = 0.25,
                                                         stratify = y)
print("Treinaremos com %d elementos e testaremos com %d elementos" % (len(treino_x), len(teste_x)))

modelo = DecisionTreeClassifier(max_depth=2)
modelo.fit(treino_x, treino_y)
previsoes = modelo.predict(teste_x)

acuracia = accuracy_score(teste_y, previsoes) * 100
print("A acurácia do stratified foi %.2f%%" % acuracia)

Treinaremos com 7500 elementos e testaremos com 2500 elementos
A acurácia do stratified foi 76.84%


Usando cross validate com k=5. O k-fold é utilizado por padrão no cross validate.

In [7]:
from sklearn.model_selection import cross_validate

SEED = 158020
np.random.seed(SEED)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo,x,y,cv=5, return_train_score=False)
media = results["test_score"].mean()
desvio_padrao = results["test_score"].std()
print("Accuracy com cross validation, 5 = [%.2f, %.2f]" % ((media -2*desvio_padrao)*100, (media+2*desvio_padrao)*100))

Accuracy com cross validation, 5 = [75.21, 76.35]


## Aleatoriedade no cross validate

In [8]:
def imprime_resultados(results):
    media = results["test_score"].mean()
    desvio_padrao = results["test_score"].std()
    print("Accuracy médio = %.2f" % (media*100))
    print("Accuracy intervalo = [%.2f, %.2f]" % ((media -2*desvio_padrao)*100, (media+2*desvio_padrao)*100))

In [9]:
from sklearn.model_selection import cross_validate
from sklearn.model_selection import KFold

SEED = 158020
np.random.seed(SEED)

cv = KFold(n_splits=10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo,x,y,cv=cv, return_train_score=False)
imprime_resultados(results)

Accuracy médio = 75.78
Accuracy intervalo = [74.37, 77.19]


Embaralhando os dados no KFold

In [10]:
SEED = 158020
np.random.seed(SEED)

cv = KFold(n_splits=10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo,x,y,cv=cv, return_train_score=False)
imprime_resultados(results)

Accuracy médio = 75.78
Accuracy intervalo = [73.58, 77.98]


### Simular situação de azar

Pode ser "azar" ou ser uma proporção desbaçlanceada entre as classes

In [11]:
dados_azar = dados.sort_values("vendido", ascending=True)
x_azar = dados_azar[["preco", "idade_do_modelo","km_por_ano"]]
y_azar = dados_azar["vendido"]
dados_azar.head()

preco  vendido  idade_do_modelo   km_por_ano
4999  74023.29        0               12  24812.80412
5322  84843.49        0               13  23095.63834
5319  83100.27        0               19  36240.72746
5316  87932.13        0               16  32249.56426
5315  77937.01        0               15  28414.50704

Rodando dados_azar sem shuffle:

In [12]:
SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits=10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo,x_azar,y_azar,cv=cv, return_train_score=False)
imprime_resultados(results)

Accuracy médio = 57.84
Accuracy intervalo = [34.29, 81.39]


Com shuffle:

In [13]:
SEED = 301
np.random.seed(SEED)

cv = KFold(n_splits=10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo,x_azar,y_azar,cv=cv, return_train_score=False)
imprime_resultados(results)

Accuracy médio = 75.78
Accuracy intervalo = [72.30, 79.26]


## Stratified KFold
Ajuda tamém com o problema de dados desbalanceados

In [14]:
from sklearn.model_selection import StratifiedKFold

SEED = 301
np.random.seed(SEED)

cv = StratifiedKFold(n_splits=10, shuffle=True)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo,x_azar,y_azar,cv=cv, return_train_score=False)
imprime_resultados(results)

Accuracy médio = 75.78
Accuracy intervalo = [72.94, 78.62]


## Group KFold
Gerando dados aleatórios de modelos de carro para simulação de agrupamento ao usar o estimador:

In [15]:
np.random.seed(SEED)
# Gerando 10000 inteiros aleatórios entre -2 e 3 e somando com a coluna de idades do modelo
dados["modelo"] = dados["idade_do_modelo"] + np.random.randint(-2,3,10000)
# Somando com módulo do menor valor +1 para remover negativos e zeros.
dados["modelo"] = dados["modelo"] + abs(dados["modelo"].min()) + 1 
dados.head()

preco  vendido  idade_do_modelo   km_por_ano  modelo
0   30941.02        1               18  35085.22134      18
1   40557.96        1               20  12622.05362      24
2   89627.50        0               12  11440.79806      14
3   95276.14        0                3  43167.32682       6
4  117384.68        1                4  12770.11290       5

In [16]:
dados.modelo.unique()

array([18, 24, 14,  6,  5, 13, 20, 19, 15,  2, 17, 12, 11, 16,  3,  7, 21,
       23, 10,  9, 22,  8,  4,  1], dtype=int64)

In [17]:
dados.modelo.value_counts()

20    901
19    798
18    771
21    723
17    709
16    668
14    621
22    575
15    573
13    557
12    511
11    401
10    371
23    370
9     336
8     278
7     206
24    199
6     181
5     108
4      76
3      44
2      17
1       6
Name: modelo, dtype: int64

#### Validação cruzada com Group KFold

In [18]:
from sklearn.model_selection import GroupKFold

SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits=10)
modelo = DecisionTreeClassifier(max_depth=2)
results = cross_validate(modelo,x_azar,y_azar,cv=cv, groups= dados["modelo"],return_train_score=False)
imprime_resultados(results)

Accuracy médio = 75.78
Accuracy intervalo = [73.67, 77.90]


### Cross validation com standard scaler

In [19]:
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVC

SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
scaler.fit(treino_x)
treino_x_escalado = scaler.transform(treino_x)
teste_x_escalado = scaler.transform(teste_x)

modelo = SVC()
modelo.fit(treino_x_escalado, treino_y)
previsoes = modelo.predict(teste_x_escalado)

acuracia = accuracy_score(teste_y, previsoes) * 100
print ("A acurácia foi %.2f%%" % acuracia)

D:\Anaconda3\envs\aluraenv\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
D:\Anaconda3\envs\aluraenv\lib\site-packages\ipykernel_launcher.py:9: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  if __name__ == '__main__':
D:\Anaconda3\envs\aluraenv\lib\site-packages\ipykernel_launcher.py:10: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  # Remove the CWD from sys.path while we load stuff.


A acurácia foi 77.48%


In [20]:
scaler = StandardScaler()
scaler.fit(x_azar)
x_azar_escalado = scaler.transform(x_azar)

D:\Anaconda3\envs\aluraenv\lib\site-packages\sklearn\preprocessing\data.py:645: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  return self.partial_fit(X, y)
D:\Anaconda3\envs\aluraenv\lib\site-packages\ipykernel_launcher.py:3: DataConversionWarning: Data with input dtype int64, float64 were all converted to float64 by StandardScaler.
  This is separate from the ipykernel package so we can avoid doing imports until


In [21]:
from sklearn.model_selection import GroupKFold

SEED = 301
np.random.seed(SEED)

cv = GroupKFold(n_splits=10)
modelo = SVC()
results = cross_validate(modelo,x_azar_escalado,y_azar,cv=cv, groups= dados["modelo"],return_train_score=False)
imprime_resultados(results)

Accuracy médio = 76.71
Accuracy intervalo = [74.30, 79.12]


In [23]:
from sklearn.pipeline import Pipeline
from sklearn.exceptions import DataConversionWarning
warnings.filterwarnings(action='ignore', category=DataConversionWarning)

SEED = 301
np.random.seed(SEED)

scaler = StandardScaler()
modelo = SVC()

pipeline = Pipeline([("transformacao",scaler),("estimador",modelo)])

cv = GroupKFold(n_splits=10)
results = cross_validate(pipeline,x_azar,y_azar,cv=cv, groups= dados["modelo"],return_train_score=False)
imprime_resultados(results)

Accuracy médio = 76.68
Accuracy intervalo = [74.28, 79.08]
